In [1]:
import pickle

In [2]:
import os
from tqdm import tqdm
from typing import Dict
from collections import defaultdict
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def numLines(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1
def loadData(filename, max_points):
    file_len = numLines(filename)
    f = open(filename, 'r')
    inputs = []
    outputs = []
    for i in tqdm(range(file_len)):
        if i == max_points:
            break
        line = f.readline()
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        inputs.append(line[0])
        outputs.append(line[1])
    data = {'inputs': inputs, 'outputs': outputs}
    return data
        
def load_entity_strings(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

def get_entity_wd_id_dict(filename):
    out = {}
    f = open(filename, 'r')
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        out[line[1]] = line[0]
    return out
    

def create_filter_dict(data) -> Dict[str, int]:
    filter_dict = defaultdict(list)
    for input, output in zip(data["inputs"], data["outputs"]):
        filter_dict[input].append(output)
    return filter_dict

def getAllFilteringEntities(input, filter_dicts):
    entities = []
    splits = ['train', 'test', 'valid']
    for s in splits:
        entities.extend(filter_dicts[s][input])
    return list(set(entities))

def wikidata_link_from_id(id):
    uri = 'https://www.wikidata.org/wiki/' + id
    return uri

In [3]:
dataset_name = 'fbwq_half_lego_cr'
entity_strings = load_entity_strings(os.path.join("data", dataset_name, "entity_strings.txt"))

In [4]:
entity_strings_set = set(entity_strings)

In [5]:
# data = {}
# splits = ['train', 'valid', 'test']
# dataset_name = 'fbwq_half'
# for split in splits:
#     data[split] = loadData(os.path.join('data', dataset_name, split + '.txt'), -1)

In [6]:
# filter_dicts = {}
# splits = ['train', 'valid', 'test']
# for split in splits:
#     filter_dicts[split] = create_filter_dict(data[split])

In [7]:
# fname = 'scores.pickle'
fname = 'scores/scores_fbwq_half_lego_cr.pickle'
# fname = 'scores_500_base_trie|.pickle'
scores_data = pickle.load(open(fname, 'rb'))

In [14]:
scores_data['prediction_strings']

[['Jamaican English which includes Jamaican Standard English is a variety of English spoken in Jamaica',
  'Jamaican Patois, known locally as Patois and called Jamaican Creole by linguists, is an English-based creole language with West African influences spoken primarily in Jamaica',
  'Jamaican Patois, known locally as Patois and called Jamaican Cretle by linguists, is an English-based creole language with West African influences spoken primarily in Jamaica',
  'Jamaican Patois, known locally as Patois and called Jamaican Creole by linguisland in Jamaican Creole, is an English-based creole language with West African influences spoken primarily in Jamaica'],
 ['A politician is a person holding or seeking an office within a government, usually by means of an election, voted for either by people or',
  'A lawyer is a person who practices law, as a barrister, judge, attorney, counsel or solicitor',
  'An editor in the professional or traditional sense is generally an individual who makes 

In [15]:
predictions_scores_dicts = []
for pred, score in tqdm(zip(scores_data['prediction_strings'], scores_data['scores'])):
    ps_pair = (pred, score)
    # remove predictions that are not entities
    ps_pairs = [ps_pair]
    ps_dict_only_entities = defaultdict(list)
    for ps in ps_pairs:
        if True:
#         if ps[0] in entity_strings_set:
            ps_dict_only_entities[ps[0]] = ps[1]
    predictions_scores_dicts.append(ps_dict_only_entities)

0it [00:00, ?it/s]


TypeError: unhashable type: 'list'

In [21]:
id = 1
print(scores_data['input_strings'][id])
print(scores_data['target_strings'][id])
print(scores_data['prediction_strings'][id])
print(scores_data['scores'][id])
# print(predictions_scores_dicts[id].keys())


predict answer: James Knox Polk was the 11th President of the United States | what did NE do before he was president |
['United States Representative', 'The Governor of Tennessee is the head of government of the U', 'The Speaker of the House is the presiding officer of the chamber']
['A politician is a person holding or seeking an office within a government, usually by means of an election, voted for either by people or', 'A lawyer is a person who practices law, as a barrister, judge, attorney, counsel or solicitor', 'An editor in the professional or traditional sense is generally an individual who makes corrective changes, or edits, in the content or format of a', 'An officer is a member of an armed force or uniformed service who holds a position of authority']
tensor([-0.0164, -0.0559, -0.1118, -0.1570])


In [17]:
predictions_scores_dicts[id]

IndexError: list index out of range

In [18]:
scores_data.keys()

dict_keys(['prediction_strings', 'scores', 'target_strings', 'input_strings'])

In [19]:
import numpy as np
# no filtering for QA
predictions_filtered = predictions_scores_dicts


In [210]:
count = {}
reciprocal_ranks = 0.0
k_list = [1,3,10]
for k in k_list:
    count[k] = 0
num_small_arrs = 0
for i in range(len(predictions_filtered)):
    targets = scores_data['target_strings'][i]
    ps_dict = predictions_filtered[i]
    ps_sorted = sorted(ps_dict.items(), key=lambda item: -item[1])
#     print(ps_dict)
    if len(ps_dict) == 0:
        preds = []
    else:
        preds = [x[0] for x in ps_sorted]
    pred = preds[0]
    
    if pred in targets:
        rank = preds.index(pred) + 1
        reciprocal_ranks += 1./rank
for k in k_list:
    hits_at_k = count[k]/len(predictions_filtered)
    print('hits@{}'.format(k), hits_at_k)
print('mrr', reciprocal_ranks/len(predictions_filtered))
print(num_small_arrs/len(predictions_filtered), 'were <10 length preds array without answer')

ZeroDivisionError: division by zero

In [211]:
id = 114
inputs = scores_data['input_strings'][id]
preds = predictions_filtered[id]
preds = sorted(preds.items(), key=lambda item: -item[1])
target = scores_data['target_strings'][id]
print(inputs, ', Target:', target)
preds[0], target

IndexError: list index out of range

In [50]:
def getHeadEntity(input):
    x = input.split(':')[1][1:]
    ent = x.split('|')[0][:-1]
    return ent

In [51]:
# see if answer is head entity
count = 0
for id in range(len(predictions_filtered)):
    input = scores_data['input_strings'][id]
    head = getHeadEntity(input)
    pred = scores_data['prediction_strings'][id]
    if head == pred:
        count += 1
        print(input)
count/len(predictions_filtered)

0.0

In [113]:
# only head/tails
count = 0
for id in range(60,120, 2):
    inputs = scores_data['input_strings'][id]
    preds = predictions_filtered[id]
    preds = sorted(preds.items(), key=lambda item: -item[1])
    target = scores_data['target_strings'][id]
    pred1 = preds[0][0]
    if pred1 == target:
        print(int(id/2), inputs, pred1)
        count += 1
'count', count

('count', 0)

In [419]:
%%html
print("<a href='your_url_here'>Showing Text</a>")


In [364]:
e2wdid['pakistan']

'Q4121082'

In [174]:
sequences = ['english', 'english language', 'french']
t = Trie(sequences)

In [178]:
t.get('x')

[]